In [2]:
# pip install pillow
import PIL
from PIL import Image, ImageEnhance
import numpy as np
import os

# 1. Define image processing function
def process_image(input_image_path, output_folder):
    # Get output image filename
    image_name = os.path.basename(input_image_path)
    output_image_path = os.path.join(output_folder, image_name)

    # Open image and convert to numpy array
    img = Image.open(input_image_path).convert('RGBA')
    img_np = np.array(img)

    # Find boundaries of non-zero pixels
    non_zero_indices = np.nonzero(img_np[:, :, :3])  # Consider only RGB channels, ignore alpha channel
    if non_zero_indices[0].size == 0 or non_zero_indices[1].size == 0:  # If no non-zero pixels exist
        new_img = Image.new('RGBA', (500, 500), (0, 0, 0, 0))  # Create a fully transparent 500*500 image
        # 500*500 is manually determined and can be adjusted based on actual cuttlefish size
    else:
        ymin, ymax = non_zero_indices[0].min(), non_zero_indices[0].max()
        xmin, xmax = non_zero_indices[1].min(), non_zero_indices[1].max()

        # Crop image
        cropped_img = img.crop((xmin, ymin, xmax + 1, ymax + 1))

        # Create 500*500 pixel transparent image
        new_img = Image.new('RGBA', (500, 500), (0, 0, 0, 0))

        # Calculate position to paste cropped image at center of new image
        left = (500 - cropped_img.width) // 2
        top = (500 - cropped_img.height) // 2

        # Paste cropped image at center of new image
        new_img.paste(cropped_img, (left, top))

        # Convert new image to numpy array for pixel value modification
        new_img_np = np.array(new_img)

        # Traverse each pixel of new image, converting completely black regions to transparent
        for i in range(new_img_np.shape[0]):
            for j in range(new_img_np.shape[1]):
                if (new_img_np[i, j, :3] == [0, 0, 0]).all():  # If RGB values are (0, 0, 0)
                    new_img_np[i, j, 3] = 0  # Set alpha to 0

        # Convert modified numpy array back to PIL image
        new_img = Image.fromarray(new_img_np)

        # Convert image to grayscale
        new_img = new_img.convert('LA')  # 'LA' mode represents grayscale with alpha channel

    # Rotate image clockwise by 0 degrees
    # rotated_img = new_img.rotate(-0, expand=True, resample=Image.BICUBIC)

    # Save result image
    rotated_img.save(output_image_path)


# 2. Traverse png files in folder and invoke image processing function
def process_folder(input_folder, output_folder):
    # Ensure output folder exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Traverse all files in input folder
    for file_name in os.listdir(input_folder):
        if file_name.endswith('.png'):  # Process only png files
            input_image_path = os.path.join(input_folder, file_name)
            process_image(input_image_path, output_folder)


# 3. Invoke function to process images in folder
input_folder = 'C:/Users/12152/Desktop/data/sepia4/vision_white_after/segmentation_test'  # Replace with actual input folder path
output_folder = 'C:/Users/12152/Desktop/data/sepia4/vision_white_after/segmentation_cut_test'  # Replace with actual output folder path
process_folder(input_folder, output_folder)

ModuleNotFoundError: No module named 'PIL'